In [1]:
import sys
import json
sys.path.append('..')
sys.path.append('../questions_construction')

from src.common import *
from model_performances import *
from copy import deepcopy

EVERYTHING_POSTFIX = 'everything'

In [2]:
def across_domains(ramifications, stats_type, model_name, eval_type):
    if ramifications == WITH_RAMIFICATIONS:
        results_path = f'{PROJECT_PATH}/results_ramifications' # todo rn to results dir
    elif ramifications == WITHOUT_RAMIFICATIONS:
        results_path = RESULTS_PATH
    else:
        raise f"{ramifications} is not defined"
        
    all_stats = []
    all_domains_qa_objects = []
    for domain in DOMAIN_NAMES:
        meta_info = {MODEL_NAME_KEY: model_name, EVAL_TYPE_KEY: eval_type, DOMAIN_KEY: domain}
        try:
            results_path = f'{results_path}/{model_name}/{eval_type}/{domain}.jsonl'
            qa_objects = open_jsonl(results_path)
            all_domains_qa_objects.extend(qa_objects)
            if stats_type==TRUE_FALSE_ANSWER:
                all_stats.append(TrueFalseStats.stats_qa_objects_ls(qa_objects) | deepcopy(meta_info))
            elif stats_type==FREE_ANSWER:
                all_stats.append(FreeAnswerStats.stats_qa_objects_ls(qa_objects) | deepcopy(meta_info))
            else:
                raise f"{stats_type} is not defined"
        except Exception as e:
            print(e)
            all_stats.append({TRUE_FALSE_ANSWER: {}, FREE_ANSWER: {}} | deepcopy(meta_info)) #TODO fix
    meta_info[DOMAIN_KEY] = ALL_DOMAINS_KEY
    if stats_type==TRUE_FALSE_ANSWER:
        all_stats.append(TrueFalseStats.stats_qa_objects_ls(all_domains_qa_objects) | deepcopy(meta_info))
    elif stats_type==FREE_ANSWER:
        all_stats.append(FreeAnswerStats.stats_qa_objects_ls(all_domains_qa_objects) | deepcopy(meta_info))
    return all_stats

def main(ramifications, stats_type):
    save_dir = os.path.join(STATISTICS_PATH, ramifications)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    all_stats = []
    for model_name in PROMPT_MODEL_NAMES:
        for eval_type in PROMPT_EVAL_TYPES:
            cross_domains_stats = across_domains(ramifications, stats_type, model_name, eval_type)
            all_stats.extend(cross_domains_stats)
            
            save_dir_combo = f'{save_dir}/{model_name}/{eval_type}' #f1_rouge_everything.json'
            if not os.path.exists(save_dir_combo):
                os.makedirs(save_dir_combo)
            with open(f'{save_dir_combo}/{stats_type}.json', 'w') as f:
                json.dump(cross_domains_stats, f)
                
    # save_path = f'{save_dir}/{stats_type}.{EVERYTHING_POSTFIX}.json' #f1_rouge_everything.json'
    # with open(save_path, 'w') as f:
    #     json.dump(all_stats, f)
    return all_stats

In [3]:
tf_stats = main(WITHOUT_RAMIFICATIONS, TRUE_FALSE_ANSWER)

[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gemini/few_shot_4/blocksworld.jsonl/gemini/few_shot_4/depots.jsonl'
[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gemini/few_shot_4/blocksworld.jsonl/gemini/few_shot_4/depots.jsonl/gemini/few_shot_4/driverlog.jsonl'
[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gemini/few_shot_4/blocksworld.jsonl/gemini/few_shot_4/depots.jsonl/gemini/few_shot_4/driverlog.jsonl/gemini/few_shot_4/goldminer.jsonl'
[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gemini/few_shot_4/blocksworld.jsonl/gemini/few_shot_4/depots.jsonl/gemini/few_shot_4/driverlog.jsonl/gemini/few_shot_4/goldminer.jsonl/gemini/few_shot_4/grippers.jsonl'
[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actio

/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/an

[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/blocksworld.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/blocksworld.jsonl/gpt4/few_shot_4/depots.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/blocksworld.jsonl/gpt4/few_shot_4/depots.jsonl/gpt4/few_shot_4/driverlog.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/blocksworld.jsonl/gpt4/few_shot_4/depots.jsonl/gpt4/few_shot_4/driverlog.jsonl/gpt4/few_shot_4/goldminer.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/blocksworld.jsonl/gpt4/few_shot_4/depots.jsonl/gpt4/few_shot_4/driverlo

/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/an

TypeError: list indices must be integers or slices, not str

In [4]:
tf_stats = main(WITH_RAMIFICATIONS, TRUE_FALSE_ANSWER)

/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/an

[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results_ramifications/gemini/few_shot_4/blocksworld.jsonl/gemini/few_shot_4/depots.jsonl'
[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results_ramifications/gemini/few_shot_4/blocksworld.jsonl/gemini/few_shot_4/depots.jsonl/gemini/few_shot_4/driverlog.jsonl'
[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results_ramifications/gemini/few_shot_4/blocksworld.jsonl/gemini/few_shot_4/depots.jsonl/gemini/few_shot_4/driverlog.jsonl/gemini/few_shot_4/goldminer.jsonl'
[Errno 20] Not a directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results_ramifications/gemini/few_shot_4/blocksworld.jsonl/gemini/few_shot_4/depots.jsonl/gemini/few_shot_4/driverlog.jsonl/gemini/few_shot_4/goldminer.jsonl/gemini/few_shot_4/grippers.jsonl'
[Errno 20] Not a directory: '/Use

/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/an